In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# 데이터 로드
df = pd.read_csv('시군구별전력사용량.csv', encoding='cp949')  # encoding='cp949' 추가

# 필요한 열 선택 및 조건 필터링
df = df[['시도', '시군구', '계약종별', '1월', '2월', '3월', '4월', '5월']]
df = df[df['계약종별'] == '합 계']

# 결과 저장할 리스트
results = []

for index, row in df.iterrows():
    # 1월부터 4월까지 데이터를 사용하여 5월 예측
    X = [[1], [2], [3], [4]]  # 월
    y = [row['1월'], row['2월'], row['3월'], row['4월']]
    model = LinearRegression()
    model.fit(X, y)

    # 5월 예측
    predicted_may = model.predict([[5]])[0]

    # 실제 5월 값
    actual_may = row['5월']

    # 결과 저장
    results.append({
        '시도': row['시도'],
        '시군구': row['시군구'],
        '예측값': np.floor(predicted_may),
        '실제값': actual_may,
        '오차': np.floor(abs(predicted_may - actual_may))
    })

# 결과를 DataFrame으로 변환
result_df = pd.DataFrame(results)

# 결과 출력 또는 저장
result_df
# result_df.to_csv('prediction_results.csv', index=False)